# Final Project: Data Cleaning

## Importing Libraries

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import *
import calendar
from tqdm import tqdm_notebook as tqdm

In [18]:
from google.colab import drive 
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

## Importing Datasets

### Load Beer Review Data

In [20]:
data_path = "/content/drive/My Drive/2021/6.859/Assignment4/beer_review_address_cleaned.csv"
beer_review_raw = pd.read_csv(data_path)
beer_review_raw.head()

,Unnamed: 0,brewery_id,brewery_name,beer_style,beer_name,beer_abv,review_overall,review_aroma,review_appearance,review_palate,review_taste,lat,long,full_address,address_3,address_2,address_1,country
0,0,0,Vecchio Birraio,English Strong Ale,Red Moon,6.2,3.000000,2.500000,3.000000,3.000000,3.0,45.550812,11.828476,"Via Caselle, 87, 35010 Campo San Martino PD, I...",Campo San Martino,Provincia di Padova,Veneto,Italy
1,1,0,Vecchio Birraio,Hefeweizen,Sausa Weizen,5.0,1.500000,2.000000,2.500000,1.500000,1.5,45.550812,11.828476,"Via Caselle, 87, 35010 Campo San Martino PD, I...",Campo San Martino,Provincia di Padova,Veneto,Italy
2,2,0,Vecchio Birraio,German Pilsener,Sausa Pils,5.0,3.000000,3.000000,3.500000,2.500000,3.0,45.550812,11.828476,"Via Caselle, 87, 35010 Campo San Martino PD, I...",Campo San Martino,Provincia di Padova,Veneto,Italy
3,3,0,Vecchio Birraio,Foreign / Export Stout,Black Horse Black Beer,6.5,3.000000,2.500000,3.000000,3.000000,3.0,45.550812,11.828476,"Via Caselle, 87, 35010 Campo San Martino PD, I...",Campo San Martino,Provincia di Padova,Veneto,Italy
4,4,1,Caldera Brewing Company,American IPA,Caldera IPA,6.1,4.055118,3.973753,4.070866,3.925197,4.0,42.181944,-122.662778,"590 Clover Ln, Ashland, OR 97520, USA",NaN,Jackson County,Oregon,United States


In [21]:
df = beer_review_raw.copy()

In [22]:
df['country'].value_counts()

United States          29471
Canada                  3038
Germany                 1828
United Kingdom          1820
Belgium                 1504
                       ...  
Dominica                   1
U.S. Virgin Islands        1
Lesotho                    1
Nepal                      1
Malaysia                   1
Name: country, Length: 123, dtype: int64

In [23]:
t = df['beer_style']

In [24]:
styles = t.to_numpy().tolist()

In [25]:
styles = set(styles)


In [26]:
style_dict = {'Ale': {'Pale Ale':{'IPA':[],
                                  'American':[],
                                  'English':[],
                                  'Belgian':[], 
                                  'Else':[]},
                      'Dark/Black/Brown Ale':{'American':[],
                                              'English':[],
                                              'Belgian':[]}, 
                      'Red Ale':[],
                      'Strong Ale':[],
                      'Else':[]}, 
              'Lager': {'Pilsner':[],
                        'Dark Lager':[],
                        'Pale/Light Lager':[],
                        'Else':[]}, 
              'Stout': {'Imperial Stout':[],
                        'Else':[]}, 
              'Porter': {'Else':[]}, 
              'Else': {'By Country': {'American':[],
                                      'English':[],
                                      'German':[],
                                      'Belgian':[],
                                      'Else':[]},
                       'High/Low ABV':[],
                       'Ingredients':{'Rye':[], 
                                      'Wheat':[], 'Else':[]},
                       
                       'Else':[]
                       }
              }
#Pilsener: type of pale lager
for name in styles:
  if 'Ale' in name:
    if 'Pale' in name:
      if 'IPA' in name or 'India Pale Ale' in name or 'Indian Pale Ale' in name:
        style_dict['Ale']['Pale Ale']['IPA'].append(name)
      elif 'American' in name:
        style_dict['Ale']['Pale Ale']['American'].append(name)
      elif 'Belgian' in name:
        style_dict['Ale']['Pale Ale']['Belgian'].append(name)
      elif 'English' in name:
        style_dict['Ale']['Pale Ale']['English'].append(name)
    elif 'Dark' in name or 'Brown' in name or 'Black' in name:
      if 'American' in name:
        style_dict['Ale']['Dark/Black/Brown Ale']['American'].append(name)
      elif 'Belgian' in name:
        style_dict['Ale']['Dark/Black/Brown Ale']['Belgian'].append(name)
      elif 'English' in name:
        style_dict['Ale']['Dark/Black/Brown Ale']['English'].append(name)
    elif 'Red' in name:
      style_dict['Ale']['Red Ale'].append(name)
    elif 'Strong' in name:
      style_dict['Ale']['Strong Ale'].append(name)
    else:
      style_dict['Ale']['Else'].append(name)
  elif 'Lager' in name:
    if 'Dark' in name or 'Dunkel' in name:
      style_dict['Lager']['Dark Lager'].append(name)
    elif 'Pale' in name or 'Light' in name or 'Helles' in name:
      style_dict['Lager']['Pale/Light Lager'].append(name)
    else:
      style_dict['Lager']['Else'].append(name)
  elif 'Stout' in name:
    if 'Imperial' in name:
      style_dict['Stout']['Imperial Stout'].append(name)
    else:
      style_dict['Stout']['Else'].append(name)
  elif 'Porter' in name:
    style_dict['Porter']['Else'].append(name)
  elif 'Pilsner' in name or 'Pilsener' in name:
    style_dict['Lager']['Pilsner'].append(name)
  elif 'IPA' in name:
    style_dict['Ale']['Pale Ale']['IPA'].append(name)
  else:
    if 'American' in name or 'California' in name:
      style_dict['Else']['By Country']['American'].append(name)
    elif 'English' in name:
      style_dict['Else']['By Country']['English'].append(name)
    elif 'German' in name or 'Gose' in name or 'Weissbier' in name or 'Altbier' in name or 'Weizenbock' in name or 'Kölsch' in name or 'Dunkelweizen' in name:
      style_dict['Else']['By Country']['German'].append(name)
    elif 'Quadrupel' in name or 'Tripel' in name or 'Low' in name or 'Doppelbock' in name or 'Dubbel' in name:
      style_dict['Else']['High/Low ABV'].append(name)
    elif 'ESB' in name:
      style_dict['Ale']['Pale Ale']['English'].append(name)
    elif 'Märzen' in name:
       style_dict['Lager']['Else'].append(name)
    elif 'Gueuze' in name or 'Faro' in name or 'Lambic' in name or 'Flanders Oud Bruin' in name:
       style_dict['Else']['By Country']['Belgian'].append(name)
    elif 'Maibock' in name:
      style_dict['Lager']['Pale/Light Lager'].append(name)
    elif 'Schwarzbier' in name:
      style_dict['Lager']['Dark Lager'].append(name)
    elif 'Sahti' in name:
      style_dict['Ale']['Else'].append(name)
    elif 'Keller Bier' in name or 'Eisbock' in name:
      style_dict['Lager']['Else'].append(name)
    elif 'Fruit' in name or 'Wheat' in name or 'Hefeweizen' in name or 'Weizen' in name or 'Rye' in name or 'Kristalweizen' in name or 'Kvass' in name or 'Witbier' in name or 'Herbed' in name or 'Roggenbier' in name:
      if 'Wheat' in name or 'Weizen' in name or 'Wit' in name or 'weizen' in name:
        style_dict['Else']['Ingredients']['Wheat'].append(name)
      elif 'Rye' in name or 'Roggenbier' in name or 'Kvass' in name:
        style_dict['Else']['Ingredients']['Rye'].append(name)
      else:
        style_dict['Else']['Ingredients']['Else'].append(name)
    elif 'Bock' in name:
      style_dict['Lager']['Else'].append(name)
    elif 'Bière de Garde' in name:
      style_dict['Ale']['Pale Ale']['Else'].append(name)
    elif 'Happoshu' in name or 'Chile' in name:
      style_dict['Else']['By Country']['Else'].append(name)
    else:
      style_dict['Else']['Else'].append(name)

In [27]:
pretty(style_dict)

Ale
	Pale Ale
		IPA
			['American IPA', 'Belgian IPA', 'American Double / Imperial IPA', 'English India Pale Ale (IPA)']
		American
			['American Pale Wheat Ale', 'American Pale Ale (APA)']
		English
			['English Pale Mild Ale', 'Extra Special / Strong Bitter (ESB)', 'English Pale Ale']
		Belgian
			['Belgian Strong Pale Ale', 'Belgian Pale Ale']
		Else
			['Bière de Garde']
	Dark/Black/Brown Ale
		American
			['American Black Ale', 'American Dark Wheat Ale', 'American Brown Ale']
		English
			['English Dark Mild Ale', 'English Brown Ale']
		Belgian
			['Belgian Dark Ale', 'Belgian Strong Dark Ale']
	Red Ale
		['American Amber / Red Ale', 'Irish Red Ale', 'Flanders Red Ale']
	Strong Ale
		['American Strong Ale', 'English Strong Ale']
	Else
		['American Blonde Ale', 'Scottish Gruit / Ancient Herbed Ale', 'Pumpkin Ale', 'Cream Ale', 'Sahti', 'Scottish Ale', 'Scotch Ale / Wee Heavy', 'American Wild Ale', 'Saison / Farmhouse Ale', 'Old Ale']
Lager
	Pilsner
		['American Double / Imperial Pi

In [28]:
import json
data_path = "/content/drive/My Drive/2021/6.859/Assignment4/beer.json"
with open(data_path) as json_file:
    json_data = json.load(json_file)

In [ ]:
count = beer_review_raw['beer_style'].value_counts()
count_dict = count.to_dict()
count_dict

In [ ]:
def recurse(json_data):
  name = None
  for key in json_data:
    if key == 'name':
      name = json_data[key] 
    elif key == 'children':
      for child in json_data['children']:
        recurse(child)
    elif key == 'value':
      json_data['value'] = count_dict[json_data['name']]
      print(json_data)
recurse(json_data)

In [31]:
dict_to_json = json.dumps(json_data)

In [32]:
dict_to_json

'{"name": "Beer", "children": [{"name": "Ale", "children": [{"name": "Pale Ale", "children": [{"name": "IPA", "children": [{"name": "English India Pale Ale (IPA)", "value": 381}, {"name": "Belgian IPA", "value": 230}, {"name": "American IPA", "value": 2211}, {"name": "American Double / Imperial IPA", "value": 1176}]}, {"name": "American", "children": [{"name": "American Pale Wheat Ale", "value": 573}, {"name": "American Pale Ale (APA)", "value": 1890}]}, {"name": "English", "children": [{"name": "English Pale Ale", "value": 872}, {"name": "Extra Special / Strong Bitter (ESB)", "value": 625}, {"name": "English Pale Mild Ale", "value": 105}]}, {"name": "Belgian", "children": [{"name": "Belgian Pale Ale", "value": 1019}, {"name": "Belgian Strong Pale Ale", "value": 712}]}, {"name": "Else", "children": [{"name": "Bi\\u00e8re de Garde", "value": 204}]}]}, {"name": "Dark/Black/Brown Ale", "children": [{"name": "American", "children": [{"name": "American Black Ale", "value": 331}, {"name": "A

In [62]:
flattened = {}

flattened['Ale'] = ['American IPA', 'Belgian IPA', 'American Double / Imperial IPA', 'English India Pale Ale (IPA)', 'American Pale Wheat Ale', 'American Pale Ale (APA)', 'English Pale Mild Ale', 'Extra Special / Strong Bitter (ESB)', 'English Pale Ale', 'Belgian Strong Pale Ale', 'Belgian Pale Ale', 'Bière de Garde', 'American Black Ale', 'American Dark Wheat Ale', 'American Brown Ale', 'English Dark Mild Ale', 'English Brown Ale', 'Belgian Dark Ale', 'Belgian Strong Dark Ale', 'American Amber / Red Ale', 'Irish Red Ale', 'Flanders Red Ale', 'American Strong Ale', 'English Strong Ale', 'American Blonde Ale', 'Scottish Gruit / Ancient Herbed Ale', 'Pumpkin Ale', 'Cream Ale', 'Sahti', 'Scottish Ale', 'Scotch Ale / Wee Heavy', 'American Wild Ale', 'Saison / Farmhouse Ale', 'Old Ale']
flattened['Lager'] = ['American Double / Imperial Pilsner', 'Czech Pilsener', 'German Pilsener', 'Euro Dark Lager', 'Schwarzbier', 'Munich Dunkel Lager', 'Light Lager', 'Munich Helles Lager', 'Euro Pale Lager', 'American Pale Lager', 'Maibock / Helles Bock', 'Eisbock', 'Euro Strong Lager', 'Bock', 'Vienna Lager', 'Keller Bier / Zwickel Bier', 'Japanese Rice Lager', 'American Adjunct Lager', 'Dortmunder / Export Lager', 'Märzen / Oktoberfest', 'American Amber / Red Lager']
flattened['Stout'] = ['American Double / Imperial Stout', 'Russian Imperial Stout', 'Irish Dry Stout', 'American Stout', 'Oatmeal Stout', 'English Stout', 'Milk / Sweet Stout', 'Foreign / Export Stout']
flattened['Porter'] = ['American Porter', 'Baltic Porter', 'English Porter']
flattened['Else'] = ['California Common / Steam Beer', 'American Malt Liquor', 'American Barleywine', 'English Barleywine', 'English Bitter', 'Gose', 'Weizenbock', 'Kölsch', 'Dunkelweizen', 'Altbier', 'Berliner Weissbier', 'Gueuze', 'Faro', 'Lambic - Unblended', 'Flanders Oud Bruin', 'Lambic - Fruit', 'Chile Beer', 'Happoshu', 'Quadrupel (Quad)', 'Low Alcohol Beer', 'Dubbel', 'Doppelbock', 'Tripel', 'Rye Beer', 'Roggenbier', 'Kvass', 'Witbier', 'Kristalweizen', 'Wheatwine', 'Hefeweizen', 'Herbed / Spiced Beer', 'Fruit / Vegetable Beer', 'Winter Warmer', 'Braggot', 'Smoked Beer', 'Rauchbier', 'Bière de Champagne / Bière Brut', 'Black & Tan']


34
21
8
3
38


In [70]:
df2 = beer_review_raw.copy()
df2['beer_class'] = None

In [71]:
for i, row in df2.iterrows():
  for name in flattened:
    if row['beer_style'] in flattened[name]:
      df2.at[i,'beer_class'] = name
      break

In [72]:
df2

,Unnamed: 0,brewery_id,brewery_name,beer_style,beer_name,beer_abv,review_overall,review_aroma,review_appearance,review_palate,review_taste,lat,long,full_address,address_3,address_2,address_1,country,beer_class
0,0,0,Vecchio Birraio,English Strong Ale,Red Moon,6.2,3.000000,2.500000,3.000000,3.000000,3.000000,45.550812,11.828476,"Via Caselle, 87, 35010 Campo San Martino PD, I...",Campo San Martino,Provincia di Padova,Veneto,Italy,Ale
1,1,0,Vecchio Birraio,Hefeweizen,Sausa Weizen,5.0,1.500000,2.000000,2.500000,1.500000,1.500000,45.550812,11.828476,"Via Caselle, 87, 35010 Campo San Martino PD, I...",Campo San Martino,Provincia di Padova,Veneto,Italy,Else
2,2,0,Vecchio Birraio,German Pilsener,Sausa Pils,5.0,3.000000,3.000000,3.500000,2.500000,3.000000,45.550812,11.828476,"Via Caselle, 87, 35010 Campo San Martino PD, I...",Campo San Martino,Provincia di Padova,Veneto,Italy,Lager
3,3,0,Vecchio Birraio,Foreign / Export Stout,Black Horse Black Beer,6.5,3.000000,2.500000,3.000000,3.000000,3.000000,45.550812,11.828476,"Via Caselle, 87, 35010 Campo San Martino PD, I...",Campo San Martino,Provincia di Padova,Veneto,Italy,Stout
4,4,1,Caldera Brewing Company,American IPA,Caldera IPA,6.1,4.055118,3.973753,4.070866,3.925197,4.000000,42.181944,-122.662778,"590 Clover Ln, Ashland, OR 97520, USA",NaN,Jackson County,Oregon,United States,Ale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43131,44064,5152,Wissey Valley Brewery,English Pale Ale,Stoked Up,4.7,2.000000,3.000000,4.000000,2.500000,2.500000,52.559611,0.579850,"River Wissey, United Kingdom",NaN,Norfolk,England,United Kingdom,Ale
43132,44065,5152,Wissey Valley Brewery,English Porter,Cherry Porter,4.7,3.965517,3.775862,4.086207,3.862069,3.931034,52.559611,0.579850,"River Wissey, United Kingdom",NaN,Norfolk,England,United Kingdom,Porter
43133,44066,5152,Wissey Valley Brewery,English Stout,Cherry Pickers Stout,6.0,2.500000,3.000000,4.000000,3.000000,4.000000,52.559611,0.579850,"River Wissey, United Kingdom",NaN,Norfolk,England,United Kingdom,Stout
43134,44073,5154,Georg Meinel Bierbrauerei KG,Weizenbock,Meinel Weizenbock,6.5,3.500000,4.000000,3.500000,3.500000,4.000000,50.326251,11.917190,"Alte Plauener Str. 24, 95028 Hof, Germany",NaN,Oberfranken,Bayern,Germany,Else


In [73]:
from google.colab import files
df2.to_csv('beer_review_address_cleaned_class.csv') 
files.download('beer_review_address_cleaned_class.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>